In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-bigdata-ChallengeD2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-29 19:22:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-07-29 19:22:29 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



LOAD AMAZON DATA INTO SPARK DATAFRAME

In [14]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(15)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

Deliverable 2

In [15]:
# Confirm total_vote, helpful_votes, and star_rating are all integers
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [16]:
#1.Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count 
# is equal to or greater than 20 to pick reviews
total_votes = df.filter("total_votes>=20")
total_votes.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [17]:
#2.Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve 
# all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_50_df = total_votes.filter("helpful_votes/total_votes>= .50")
helpful_50_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [18]:
print(helpful_50_df.count())

38010


In [19]:
#3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written 
# as part of the Vine program (paid), vine == 'Y'
vine_review = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
vine_review.show()
print(vine_review.count())

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44759540|R3A71VR1JZD8WF|B00JGWN8O8|     378897034|Petcube Interacti...|    Pet Products|          2|           27|         30|   Y|                N|Great idea, not s...|I do not hate thi...| 2015-08-29|
|         US|   37388532|R16OMUJIGI18JZ|B00IP05CUA|     833937853|Wellness Kittles ...|    Pet Products|          5|    

In [20]:
#4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
novine_review = helpful_50_df.filter(helpful_50_df["vine"] == "N")
novine_review.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [21]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews 
# for the two types of review (paid vs unpaid).
vine_review_count = vine_review.count()
print("Total Number of Reviews PAID and 'helpful': %f" % vine_review_count)

novine_review_count = novine_review.count()
print("Total Number of Reviews NON-PAID and 'helpful': %f" % novine_review_count)



Total Number of Reviews PAID and 'helpful': 170.000000
Total Number of Reviews NON-PAID and 'helpful': 37840.000000


In [25]:
# Number of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_df = vine_review.filter(vine_review['star_rating'] == "5")
print("Total Number of 5-star Reviews PAID and 'helpful': %f" % star5_vine_df.count())

star5_novine_df = novine_review.filter(novine_review["star_rating"]=="5")
print("Total Number of 5-star Reviews NON-PAID and 'helpful': %f" % star5_novine_df.count())


Total Number of 5-star Reviews PAID and 'helpful': 65.000000
Total Number of 5-star Reviews NON-PAID and 'helpful': 20612.000000


In [26]:
# Percentage of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_percent = (star5_vine_df.count()/vine_review_count)
print("Percent of 5-Star Pet Products Reviews from PAID, 'helpful' dataset: %f" % star5_vine_percent)

star5_novine_percent = (star5_novine_df.count()/novine_review_count)
print("Percent of 5-Star Pet Products Reviews from NON-PAID, 'helpful' dataset: %f" % star5_novine_percent)

Percent of 5-Star Pet Products Reviews from PAID, 'helpful' dataset: 0.382353
Percent of 5-Star Pet Products Reviews from NON-PAID, 'helpful' dataset: 0.544715


In [28]:
#Total number of ALL reviews
total_review_counts = df.count()
print("Total Number of Pet Products Reviews: %f" % total_review_counts)

helpful_review_counts = helpful_50_df.count()
print("Total Number of 'Helpful' Pet Products Reviews: %f" % helpful_review_counts)

Total Number of Pet Products Reviews: 2643619.000000
Total Number of 'Helpful' Pet Products Reviews: 38010.000000


In [30]:
#Total number of ALL 5-star reviews
star5_df = df.filter(df['star_rating']=='5')
star5_df.show()

star5_count = star5_df.count()
print("Total Number of 5-Star Pet Products Reviews: %f" % star5_count)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   43214993|R14QJW3XF8QO1P|B0084OHUIO|     902215727|Tyson's True Chew...|    Pet Products|          5|    

In [31]:
#Total number of 5-star HELPFUL reviews
star5_helpful_df = helpful_50_df.filter(helpful_50_df['star_rating']=='5')
star5_helpful_df.show(10)

star5_helpful_count = star5_helpful_df.count()
print("Total Number of 5-Star 'Helpful' Pet Products Reviews: %f" % star5_helpful_count)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|           25|         25|   N|                Y|Two seconds flat ...|Once I set it up,...| 2015-08-31|
|         US|   48415890|R3VQPJZ54B55BA|B0108LVGZI|     301347908|Medicated Oatmeal...|    Pet Products|          5|    

In [32]:
# Percentage of ALL 5-star reviews paid vs non-paid 
# 5-star Paid reviews 
vine_star5_all_df = star5_df.filter(star5_df['vine'] == 'Y')
vine_star5_all_count = vine_star5_all_df.count()
vine_star5_all_percent = (vine_star5_all_count/star5_count) 

print("Number of 5-Star Pet Products Reviews PAID: %f" % vine_star5_all_count)
print("Percent of 5-Star Pet Products Reviews PAID: %f" % vine_star5_all_percent)

#5-star Non-Paid reviews
novine_star5_all_df = star5_df.filter(star5_df['vine'] == 'N')
novine_star5_all_count = novine_star5_all_df.count()
novine_star5_all_percent = (novine_star5_all_count/star5_count)

print("Number of 5-Star Pet Products Reviews NON-PAID: %f" % novine_star5_all_count)
print("Percent of 5-Star Pet Products Reviews NON-PAID: %f" % novine_star5_all_percent)

Number of 5-Star Pet Products Reviews PAID: 4343.000000
Percent of 5-Star Pet Products Reviews PAID: 0.002639
Number of 5-Star Pet Products Reviews NON-PAID: 1641210.000000
Percent of 5-Star Pet Products Reviews NON-PAID: 0.997361


In [33]:
# Percentage of HELPFUL 5-star reviews paid vs non-paid (% of 5 star reviews in Step 3 dataframe) 
# 5-star Paid "helpful" reviews 
vine_star5_helpful = star5_helpful_df.filter(star5_helpful_df['vine']=='Y')
vine_star5_helpful_count = vine_star5_helpful.count()
vine_star5_helpful_percent = (vine_star5_helpful_count/star5_helpful_count)

print("Number of 5-Star, 'Helpful' Pet Products Reviews PAID: %f" % vine_star5_helpful_count)
print("Percent of 5-Star,'Helpful'Pet Products Reviews PAID: %f" % vine_star5_helpful_percent)

#5-star NON-Paid "helpful" reviews
novine_star5_helpful = star5_helpful_df.filter(star5_helpful_df['vine'] == 'N')
novine_star5_helpful_count = novine_star5_helpful.count()
novine_star5_helpful_percent = (novine_star5_helpful_count/star5_helpful_count)

print("Number of 5-Star, 'Helpful' Pet Products Reviews NON-PAID: %f" % novine_star5_helpful_count)
print("Percent of 5-Star,'Helpful'Pet Products Reviews NON-PAID: %f" % novine_star5_helpful_percent)
 

Number of 5-Star, 'Helpful' Pet Products Reviews PAID: 65.000000
Percent of 5-Star,'Helpful'Pet Products Reviews PAID: 0.003144
Number of 5-Star, 'Helpful' Pet Products Reviews NON-PAID: 20612.000000
Percent of 5-Star,'Helpful'Pet Products Reviews NON-PAID: 0.996856


## **Additional Vine Analysis **

In [34]:
#Percent of Vine vs non-Vine all reviews
vine_all_df = df.filter(df["vine"] == "Y")
novine_all_df = df.filter(df["vine"] == "N")
vine_percent_all = (vine_all_df.count()/df.count())
novine_percent_all = (novine_all_df.count()/df.count())

print(df.count())
print(vine_all_df.count())
print(novine_all_df.count())
print(vine_percent_all)
print(novine_percent_all)

2643619
10215
2633399
0.0038640212526842937
0.9961340874006428


In [35]:
# Percent of All Vine that is 5-Star
vine_all_star5_df = vine_all_df.filter(vine_all_df["star_rating"] == '5')
vine_all_star_df = vine_all_df.filter(vine_all_df["star_rating"] != '5')

novine_all_star5_df = novine_all_df.filter(novine_all_df["star_rating"] == '5')
novine_all_star_df = novine_all_df.filter(novine_all_df["star_rating"] != '5')

vine_pct_star5_all = (vine_all_star5_df.count()/vine_all_df.count())
novine_pct_star5_all = (novine_all_star5_df.count()/novine_all_df.count())

print(vine_all_star5_df.count())
print(vine_all_star_df.count())

print(novine_all_star5_df.count())
print(novine_all_star_df.count())

print(vine_pct_star5_all)
print(novine_pct_star5_all)

4343
5872
1641210
992189
0.42515907978463047
0.6232287625232636


In [37]:
#pull all vine reviewed products 
vine_prod_df = vine_all_df.select(vine_all_df['product_id']).dropDuplicates()
print(vine_prod_df.count())

496


In [38]:
#join the vine_prod_df to df
joined_df = vine_prod_df.join(df, vine_all_df.product_id == df.product_id, how = "inner").dropDuplicates()
joined_df.show(10)
joined_df.count()

+----------+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|product_id|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+----------+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|B0096EZB96|         US|   48803746|R1FHVFU5LV1HDM|B0096EZB96|      86490977|Fancy Feast Gourm...|    Pet Products|          5|            0|          0|   N|                Y|great way to buy ...|cats like it, che...| 2014-02-04|
|B0096EZB96|         US|   53023241| RJS698A9H3UIK|B0096EZB96|      86490977

33734

In [39]:
# Rating breakdown 
new_df = joined_df.groupby("star_rating").count()
new_df.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 2793|
|          3| 3594|
|          5|18595|
|          4| 6655|
|          2| 2097|
+-----------+-----+



In [40]:
# Total Vine Reviews for Vine Products
vine_new_df = joined_df.filter(joined_df['vine']=="Y")
vine_new_df.count()

10215

In [42]:
# Total noVine Reviews for Vine Products
novine_new_df = joined_df.filter(joined_df["vine"] == "N")
novine_new_df.count()

23519

In [43]:
# count of star ratings of vine and non vine reviews
vine_star5_new = vine_new_df.groupby("star_rating").count()
vine_star5_new.show()

novine_star5_new = novine_new_df.groupby("star_rating").count()
novine_star5_new.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|  234|
|          3| 1703|
|          5| 4343|
|          4| 3302|
|          2|  633|
+-----------+-----+

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 2559|
|          3| 1891|
|          5|14252|
|          4| 3353|
|          2| 1464|
+-----------+-----+

